In [1]:
from langchain_experimental.tools import PythonREPLTool
import text

# define python repl
python_repl = PythonREPLTool()

# initiate python_repl to ignore warnings
python_repl.invoke('import warnings\nwarnings.simplefilter("ignore")')

Python REPL can execute arbitrary code. Use with caution.


''

In [2]:
from dotenv import load_dotenv, find_dotenv
import re

from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda
from langchain_community.chat_models import BedrockChat

# read local .env file
_ = load_dotenv(find_dotenv()) 

# define language model
model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
#model_id = 'anthropic.claude-3-haiku-20240307-v1:0'
llm = BedrockChat(model_id=model_id, model_kwargs={'temperature': 0})

# set a distance threshold for when to create a new plan vs modify an existing plan
threshold = .5


def extract_text_between_markers(text):
    '''Helper function to extract code'''
    start_marker = '```python'
    end_marker = '```'

    pattern = re.compile(f'{re.escape(start_marker)}(.*?){re.escape(end_marker)}', re.DOTALL)
    matches = pattern.findall(text.content)
    return matches[0]


CONVERT_SYSTEM_PROMPT = '''<instructions>You are a highly skilled Python programmer.  Your goal is to help a user execute a plan by writing code for a Python REPL.</instructions>

Text between the <function_detail></function_detail> tags is documentation on the functions in use.  Do not attempt to use any feature that is not explicitly listed in the data dictionary for that function.
<function_detail> 
{function_detail}
</function_detail>

Text between the <task></task> tags is the goal of the plan.
<task>
{task}
</task>

Text between the <plan></plan> tags is the entire plan that will be executed.
<plan>
{plan}
</plan>

Text between the <rules></rules> tags are rules that must be followed.
<rules>
1. Import all necessary libraries at the start of your code.
2. Always assign the result of a pybaseball function call to a variable.
3. Use print() when you want to display the final result to the User.
4. Never write functions
5. Return all python code between three tick marks like this:
```python
python code goes here
```
6. Comment your code liberally to be clear about what is happening and why.
</rules>
'''

In [3]:
step = '''1. Get all pitch data from 2020-08-01 to 2020-08-07:
all_pitches = statcast('2020-08-01', '2020-08-07')'''

In [4]:
convert_prompt_template = ChatPromptTemplate.from_messages([
    ("system", CONVERT_SYSTEM_PROMPT),
    MessagesPlaceholder(variable_name="messages"), 
])

convert_chain = convert_prompt_template | llm | RunnableLambda(extract_text_between_markers)

convert_prefix = 'Here is the next step of the plan.  Convert this step into code that can be executed in a Python REPL.'
messages = [HumanMessage(content=f'{convert_prefix}\n\n{step}')]

# invoke convert chain
code = convert_chain.invoke({'messages':messages, 'step':step, 'plan':text.plan, 'function_detail':text.function_detail, 'task':text.task}) # , 'function_detail_str': function_detail

In [5]:
print(code)


# Import the statcast function from pybaseball
from pybaseball import statcast

# Get all pitch data from August 1st to August 7th, 2020
all_pitches = statcast('2020-08-01', '2020-08-07')



In [6]:
result = python_repl.invoke(code)

100%|██████████| 7/7 [00:04<00:00,  1.49it/s]


In [7]:
result

'This is a large query, it may take a moment to complete\n'

In [8]:
step = """2. Filter for just curveballs:
all_curves = all_pitches[all_pitches['pitch_type'] == 'CU']"""

In [9]:
messages.append(AIMessage(content=f'The previous step completed successfully with the following code:\n\n```python\n{code}\n```\n\nHere was the result: {result}'))
messages.append(HumanMessage(content=f'{convert_prefix}\n\n{step}'))

# invoke convert chain
code = convert_chain.invoke({'messages':messages, 'step':step, 'plan':text.plan, 'function_detail':text.function_detail, 'task':text.task}) # , 'function_detail_str': function_detail

In [10]:
print(code)


# Filter the all_pitches dataframe to only include curveballs
all_curves = all_pitches[all_pitches['pitch_type'] == 'CU']



In [11]:
result = python_repl.invoke(code)

In [12]:
print(result)

In [13]:
step = """3. Create a feature vector for each pitcher's curveballs:
pitcher_curves = all_curves.groupby('pitcher')
pitcher_features = pitcher_curves[['release_speed', 'release_spin', 'pfx_x', 'pfx_z']].mean().reset_index()"""

In [14]:
messages.append(AIMessage(content=f'The previous step completed successfully with the following code:\n\n```python\n{code}\n```\n\nHere was the result: {result}'))
messages.append(HumanMessage(content=f'{convert_prefix}\n\n{step}'))

# invoke convert chain
code = convert_chain.invoke({'messages':messages, 'step':step, 'plan':text.plan, 'function_detail':text.function_detail, 'task':text.task}) # , 'function_detail_str': function_detail

In [15]:
print(code)


# Group the curveball data by pitcher
pitcher_curves = all_curves.groupby('pitcher')

# For each pitcher, take the mean of the release_speed, release_spin, pfx_x, and pfx_z columns
# This creates a feature vector summarizing each pitcher's curveballs
pitcher_features = pitcher_curves[['release_speed', 'release_spin', 'pfx_x', 'pfx_z']].mean().reset_index()



In [16]:
result = python_repl.invoke(code)

In [17]:
result

'KeyError("Columns not found: \'release_spin\'")'

In [18]:
messages.append(AIMessage(content=f'The previous step reached an error with the following code:\n\n```python\n{code}\n```\n\nHere was the error: {result}'))
messages.append(HumanMessage(content=f'What information would be useful in order to troubleshoot this error?  Write Python code that can be executed in a python repl to confirm this information.'))

# invoke convert chain
code = convert_chain.invoke({'messages':messages, 'step':step, 'plan':text.plan, 'function_detail':text.function_detail, 'task':text.task}) 

In [19]:
print(code)


print(all_curves.columns)



In [20]:
result = python_repl.invoke(code)

In [21]:
print(result)

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estima

In [22]:
messages.append(AIMessage(content=f'The following code was executed to help troubleshoot this error:\n\n```python\n{code}\n```\n\nHere is the result:\n\n{result}'))
messages.append(HumanMessage(content=f'{convert_prefix}\n\n{step}'))

# invoke convert chain
code = convert_chain.invoke({'messages':messages, 'step':step, 'plan':text.plan, 'function_detail':text.function_detail, 'task':text.task}) 

In [23]:
print(code)


# Group the curveball data by pitcher 
pitcher_curves = all_curves.groupby('pitcher')

# For each pitcher, take the mean of the release_speed, release_spin_rate, pfx_x, and pfx_z columns
# This creates a feature vector summarizing each pitcher's curveballs
pitcher_features = pitcher_curves[['release_speed', 'release_spin_rate', 'pfx_x', 'pfx_z']].mean().reset_index()



In [24]:
result = python_repl.invoke(code)

In [25]:
print(result)

In [27]:
step = """4. Get Max Scherzer's player ID:
from pybaseball import playerid_lookup
scherzer_id = playerid_lookup('scherzer', 'max')['key_mlbam'][0]"""

In [28]:
messages.append(AIMessage(content=f'The previous step completed successfully with the following code:\n\n```python\n{code}\n```\n\nHere was the result: {result}'))
messages.append(HumanMessage(content=f'{convert_prefix}\n\n{step}'))

# invoke convert chain
code = convert_chain.invoke({'messages':messages, 'step':step, 'plan':text.plan, 'function_detail':text.function_detail, 'task':text.task}) # , 'function_detail_str': function_detail

In [29]:
print(code)


# Import the playerid_lookup function
from pybaseball import playerid_lookup

# Get Max Scherzer's MLBAM player ID
scherzer_id = playerid_lookup('scherzer', 'max')['key_mlbam'][0]



In [30]:
result = python_repl.invoke(code)

In [31]:
print(result)

Gathering player lookup table. This may take a moment.



In [32]:
step = """5. Get Scherzer's feature vector:
scherzer_features = pitcher_features[pitcher_features['pitcher'] == scherzer_id]"""

In [33]:
messages.append(AIMessage(content=f'The previous step completed successfully with the following code:\n\n```python\n{code}\n```\n\nHere was the result: {result}'))
messages.append(HumanMessage(content=f'{convert_prefix}\n\n{step}'))

# invoke convert chain
code = convert_chain.invoke({'messages':messages, 'step':step, 'plan':text.plan, 'function_detail':text.function_detail, 'task':text.task}) # , 'function_detail_str': function_detail

In [34]:
print(code)


# Filter the pitcher_features dataframe to only include Max Scherzer's row
scherzer_features = pitcher_features[pitcher_features['pitcher'] == scherzer_id]



In [35]:
result = python_repl.invoke(code)

In [36]:
print(result)

In [37]:
step = """6. Import scikit-learn and create a knn model:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=4)
knn.fit(pitcher_features[['release_speed', 'release_spin', 'pfx_x', 'pfx_z']])"""

In [38]:
messages.append(AIMessage(content=f'The previous step completed successfully with the following code:\n\n```python\n{code}\n```\n\nHere was the result: {result}'))
messages.append(HumanMessage(content=f'{convert_prefix}\n\n{step}'))

# invoke convert chain
code = convert_chain.invoke({'messages':messages, 'step':step, 'plan':text.plan, 'function_detail':text.function_detail, 'task':text.task}) # , 'function_detail_str': function_detail

In [39]:
print(code)


# Import the NearestNeighbors class
from sklearn.neighbors import NearestNeighbors  

# Create a kNN model to find the 4 nearest neighbors
knn = NearestNeighbors(n_neighbors=4)

# Fit the kNN model on the feature vectors
knn.fit(pitcher_features[['release_speed', 'release_spin_rate', 'pfx_x', 'pfx_z']]) 



In [40]:
result = python_repl.invoke(code)

In [41]:
print(result)

In [42]:
step = """7. Find the 3 pitchers closest to Scherzer:
distances, indices = knn.kneighbors(scherzer_features[['release_speed', 'release_spin', 'pfx_x', 'pfx_z']])
closest_indices = indices[0][1:4]
similar_pitchers = pitcher_features.iloc[closest_indices]
print(similar_pitchers[['pitcher']])'''"""

In [43]:
messages.append(AIMessage(content=f'The previous step completed successfully with the following code:\n\n```python\n{code}\n```\n\nHere was the result: {result}'))
messages.append(HumanMessage(content=f'{convert_prefix}\n\n{step}'))

# invoke convert chain
code = convert_chain.invoke({'messages':messages, 'step':step, 'plan':text.plan, 'function_detail':text.function_detail, 'task':text.task}) # , 'function_detail_str': function_detail

In [44]:
print(code)



# Find the distances and indices of the nearest neighbors to Scherzer
distances, indices = knn.kneighbors(scherzer_features[['release_speed', 'release_spin_rate', 'pfx_x', 'pfx_z']])

# Get the indices of the 2nd, 3rd, and 4th closest pitchers (since 1st is Scherzer himself)
closest_indices = indices[0][1:4]  

# Use those indices to get the rows for the similar pitchers
similar_pitchers = pitcher_features.iloc[closest_indices]

# Print out just the pitcher column which contains the player IDs
print(similar_pitchers[['pitcher']]) 




In [45]:
result = python_repl.invoke(code)

In [46]:
print(result)

     pitcher
85    593833
199   664285
122   608331



In [48]:
messages.append(AIMessage(content=f'The previous step completed successfully with the following code:\n\n```python\n{code}\n```\n\nHere was the result: {result}'))
messages.append(HumanMessage(content='Does this result give you the information needed to solve solve the task?'))



IndexError: list index out of range

In [50]:
answer_chain = convert_prompt_template | llm 
# invoke convert chain
answer = answer_chain.invoke({'messages':messages, 'step':step, 'plan':text.plan, 'function_detail':text.function_detail, 'task':text.task}) # , 'function_detail_str': function_detail

In [51]:
print(answer)

content="Yes, this result provides the information needed to solve the task of finding the 3 pitchers whose curveballs were most similar to Max Scherzer's during the first week of August 2020.\n\nThe output shows the MLBAM player IDs of the 3 pitchers whose curveball metrics (release speed, spin rate, horizontal and vertical movement) were closest to Scherzer's based on the k-nearest neighbors analysis.\n\nTo get the actual names of these 3 pitchers, we could use the playerid_lookup function again, passing in each of those 3 IDs.\n\nSo this code has successfully identified the 3 most similar curveball pitchers to Max Scherzer during that timeframe using the statcast and player lookup data from the pybaseball library combined with k-nearest neighbors from scikit-learn." response_metadata={'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0', 'usage': {'prompt_tokens': 8616, 'completion_tokens': 191, 'total_tokens': 8807}} id='run-7aa54d03-bb76-4823-92ee-13737cf39c2c-0'


In [ ]:
messages = 

In [47]:
messages

[HumanMessage(content="Here is the next step of the plan.  Convert this step into code that can be executed in a Python REPL.\n\n1. Get all pitch data from 2020-08-01 to 2020-08-07:\nall_pitches = statcast('2020-08-01', '2020-08-07')"),
 AIMessage(content="The previous step completed successfully with the following code:\n\n```python\n\n# Import the statcast function from pybaseball\nfrom pybaseball import statcast\n\n# Get all pitch data from August 1st to August 7th, 2020\nall_pitches = statcast('2020-08-01', '2020-08-07')\n\n```\n\nHere was the result: This is a large query, it may take a moment to complete\n"),
 HumanMessage(content="Here is the next step of the plan.  Convert this step into code that can be executed in a Python REPL.\n\n2. Filter for just curveballs:\nall_curves = all_pitches[all_pitches['pitch_type'] == 'CU']"),
 AIMessage(content="The previous step completed successfully with the following code:\n\n```python\n\n# Filter the all_pitches dataframe to only include 

In [49]:
messages

[HumanMessage(content="Here is the next step of the plan.  Convert this step into code that can be executed in a Python REPL.\n\n1. Get all pitch data from 2020-08-01 to 2020-08-07:\nall_pitches = statcast('2020-08-01', '2020-08-07')"),
 AIMessage(content="The previous step completed successfully with the following code:\n\n```python\n\n# Import the statcast function from pybaseball\nfrom pybaseball import statcast\n\n# Get all pitch data from August 1st to August 7th, 2020\nall_pitches = statcast('2020-08-01', '2020-08-07')\n\n```\n\nHere was the result: This is a large query, it may take a moment to complete\n"),
 HumanMessage(content="Here is the next step of the plan.  Convert this step into code that can be executed in a Python REPL.\n\n2. Filter for just curveballs:\nall_curves = all_pitches[all_pitches['pitch_type'] == 'CU']"),
 AIMessage(content="The previous step completed successfully with the following code:\n\n```python\n\n# Filter the all_pitches dataframe to only include 